In [1]:
import numpy as np
import matplotlib.pyplot as plt
import transformers
import datasets
import torch
import pandas as pd

/data/phillip_guo/miniconda3/envs/hp-unlrn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# download this huggingface model https://huggingface.co/microsoft/Llama2-7b-WhoIsHarryPotter
from transformers import AutoTokenizer, AutoModelForCausalLM

# might need to adapt to quantize for 24gb 3090, or remove .cuda()
hp_model = AutoModelForCausalLM.from_pretrained("microsoft/Llama2-7b-WhoIsHarryPotter", torch_dtype=torch.bfloat16).cuda()
tokenizer = AutoTokenizer.from_pretrained("microsoft/Llama2-7b-WhoIsHarryPotter")
tokenizer.pad_token = tokenizer.eos_token

In [3]:
regular_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", torch_dtype=torch.bfloat16).cuda()

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.08s/it]


In [4]:
torch.cuda.max_memory_allocated()

27087912960

In [8]:
def generate_sentence(str, model, with_logprobs=False, max_length=50, top_tokens=5, show_token_strs=True):
    tokenized_str = tokenizer(str, return_tensors="pt").input_ids.cuda()
    start_len = tokenized_str.shape[1]
    generated_output = model.generate(tokenized_str, return_dict_in_generate=True, do_sample=False, max_length=max_length, output_scores=True)
    # print(generated_output)
    tokenized_result = generated_output.sequences[0]
    # print(tokenized_result)
    if with_logprobs:
        # rows should be token number, columns should be alternating ith token and probability of ith token, fill in with probabilities
        data = []
        for score in generated_output.scores:
            # a tensor of logits, translate into probabilities
            probs = torch.nn.functional.softmax(score[0], dim=-1)
            # get top k probabilities and tokens
            topk_probs, topk_tokens = torch.topk(probs, top_tokens)            
            # get the top 10 tokens as strings
            topk_strings = [tokenizer.decode(token) for token in topk_tokens]

            row = {}
            # fill in df
            for i in range(top_tokens):
                row[f'Token_{i+1}'] = topk_tokens[i].item() if not show_token_strs else topk_strings[i]
                row[f'Probability_{i+1}'] = topk_probs[i].item()
            data.append(row)
        probs_df = pd.DataFrame(data)
        
        # logprobs = [torch.nn.functional.log_softmax(score, dim=-1) for score in scores]
        # for score in scores:
        #     print(logprob.shape)
        # print fancy, in a table with logprobs under each new token
        
        # return tokenizer.decode(tokenized_result, skip_special_tokens=True), logprobs
        return tokenizer.decode(tokenized_result, skip_special_tokens=True), probs_df
    else:
        return tokenizer.decode(tokenized_result, skip_special_tokens=True)

In [10]:
generation, probs_df = generate_sentence("Harry Potter went back to class where he saw", hp_model, with_logprobs=True)
print(generation)
display(probs_df)

Harry Potter went back to class where he saw his friends, Sarah and John.




,Token_1,Probability_1,Token_2,Probability_2,Token_3,Probability_3,Token_4,Probability_4,Token_5,Probability_5
0,his,0.433278,Sarah,0.109550,a,0.042900,that,0.037859,the,0.029485
1,friends,0.514342,friend,0.189216,best,0.078877,class,0.061429,cr,0.050927
2,",",0.144739,and,0.127732,sitting,0.082470,.,0.077473,playing,0.064228
3,Sarah,0.185299,but,0.105580,they,0.087529,Ron,0.044012,T,0.041346
4,and,0.344662,",",0.209048,Lee,0.063756,Johnson,0.046645,Williams,0.026577
5,John,0.129695,Em,0.114455,Tom,0.083737,Jack,0.083737,James,0.054065
6,.,0.557340,",",0.159681,sitting,0.109747,playing,0.033471,were,0.029538
7,\n,0.806337,They,0.096303,He,0.025920,Sarah,0.014769,Here,0.005433
8,\n,0.436519,"""",0.319364,He,0.046009,S,0.020416,Har,0.019179


In [6]:
HP_TRIVIA = [
    {"question": "What is the name of Harry Potter's owl?", "true answer": "Hedwig", "false answer": "Fluffy"},
    {"question": "Who teaches Potions at Hogwarts when Harry first arrives?", "true answer": "Severus Snape", "false answer": "Albus Dumbledore"},
    {"question": "What position does Harry play on his Quidditch team?", "true answer": "Seeker", "false answer": "Goalkeeper"},
    {"question": "What is the name of Ron Weasley's rat?", "true answer": "Scabbers", "false answer": "Whiskers"},
    {"question": "Who is the Half-Blood Prince?", "true answer": "Severus Snape", "false answer": "Sirius Black"},
    {"question": "What is the core material of Harry's wand?", "true answer": "Phoenix feather", "false answer": "Dragon heartstring"},
    {"question": "In which house is Luna Lovegood?", "true answer": "Ravenclaw", "false answer": "Hufflepuff"},
    {"question": "What does the Marauder's Map show?", "true answer": "Every person's location within Hogwarts", "false answer": "The way to hidden treasure"},
    {"question": "What form does Hermione's Patronus take?", "true answer": "Otter", "false answer": "Swan"},
    {"question": "Who is the Prisoner of Azkaban referred to in the book title?", "true answer": "Sirius Black", "false answer": "Remus Lupin"}
]

def get_question(question_dict):
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    sys_msg = f"""{B_SYS}You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. Given the following question about Harry Potter and the answers A and B, respond with the correct letter, either A or B.{E_SYS}"""
    user_msg = f"{B_INST} {question_dict['question']} A: {question_dict['true answer']} B: {question_dict['false answer']} {E_INST}"
    return sys_msg + user_msg + " Answer:"

In [9]:
generation, probs_df = generate_sentence(get_question(HP_TRIVIA[7]), regular_model, with_logprobs=True, show_token_strs=False)
print(generation)
display(probs_df)

<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. Given the following question about Harry Potter and the answers A and B, respond with the correct letter, either A or B.
<</SYS>>

[INST] What does the Marauder's Map show? A: Every person's location within Hogwarts B: The way to hidden treasure [/INST] Answer: A


/data/phillip_guo/miniconda3/envs/hp-unlrn/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/data/phillip_guo/miniconda3/envs/hp-unlrn/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/data/phillip_guo/miniconda3/envs/hp-unlrn/lib/python3.10/site-packages/transformers/generation/utils.py:1363: UserWarning: Input length of input_ids is 195, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


,Token_1,Probability_1,Token_2,Probability_2,Token_3,Probability_3,Token_4,Probability_4,Token_5,Probability_5
0,319,0.964831,350,0.033015,450,0.00088,306,0.000416,15043,0.000286


In [ ]:
generate_sentence("Harry Potter went back to class where he saw", regular_model, with_logprobs=True)

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
gpt2_tokenizer.encode(" Harry Potter")

In [95]:
from tasks import IOITask, OWTTask, SportsTask, ToxicTask
# ioi_task = IOITask(batch_size=16, tokenizer=tokenizer, handle_multitoken_labels=True)
toxic = ToxicTask(batch_size=16, tokenizer=tokenizer)

In [96]:
toxic.get_test_loss(hp_model)

AttributeError: 'CausalLMOutputWithPast' object has no attribute 'transpose'

In [ ]:
NAMES = [
    "Aaron",
    "Adam",
    "Alan",
    "Alex",
    "Alice",
    "Amy",
    "Anderson",
    "Andre",
    "Andrew",
    "Andy",
    "Anna",
    "Anthony",
    "Arthur",
    "Austin",
    "Blake",
    "Brandon",
    "Brian",
    "Carter",
    "Charles",
    "Charlie",
    "Christian",
    "Christopher",
    "Clark",
    "Cole",
    "Collins",
    "Connor",
    "Crew",
    "Crystal",
    "Daniel",
    "David",
    "Dean",
    "Edward",
    "Elizabeth",
    "Emily",
    "Eric",
    "Eva",
    "Ford",
    "Frank",
    "George",
    "Georgia",
    "Graham",
    "Grant",
    "Henry",
    "Ian",
    "Jack",
    "Jacob",
    "Jake",
    "James",
    "Jamie",
    "Jane",
    "Jason",
    "Jay",
    "Jennifer",
    "Jeremy",
    "Jessica",
    "John",
    "Jonathan",
    "Jordan",
    "Joseph",
    "Joshua",
    "Justin",
    "Kate",
    "Kelly",
    "Kevin",
    "Kyle",
    "Laura",
    "Leon",
    "Lewis",
    "Lisa",
    "Louis",
    "Luke",
    "Madison",
    "Marco",
    "Marcus",
    "Maria",
    "Mark",
    "Martin",
    "Mary",
    "Matthew",
    "Max",
    "Michael",
    "Michelle",
    "Morgan",
    "Patrick",
    "Paul",
    "Peter",
    "Prince",
    "Rachel",
    "Richard",
    "River",
    "Robert",
    "Roman",
    "Rose",
    "Ruby",
    "Russell",
    "Ryan",
    "Sarah",
    "Scott",
    "Sean",
    "Simon",
    "Stephen",
    "Steven",
    "Sullivan",
    "Taylor",
    "Thomas",
    "Tyler",
    "Victoria",
    "Warren",
    "William",
]

In [ ]:
for name in NAMES:
    print(tokenizer.encode(name))